# Notebook to Latex

**Overview:**  
This is an attempt to integrate a notebook chapter to latex .tex file as a sub file. Aim is to make this tex file standalone so can be compiled individually also without relying on preamble from main file. 

**Steps:**  
* Create the necessary template. Remember for the resulting tex to be standalone we need below addition apart from usual styles in preamble. 

```
\documentclass[float=false, crop=false]{standalone}
\usepackage[subpreambles=true]{standalone}
\usepackage{import}
```
* Create the tex file in ```chapters``` folder. 
* Test with running once generating pdf (should work as its also standalone). 

>This notebook sits in tools folder. Remove all additional files on successful build.

<span style="background-color: #FFFF00">If you want to remove any input part of cell to be removed during export, tag the cell as <b>'to_remove'</b></span>

**Input Parameters:**  

In [21]:
source_file = "../chapters/24_confidence_intervals_shallow_examples"  # file to be converted
# source_file = "../../24. Confidence Intervals - Deep Examples"  # file to be converted
template_file = "ipy2tex_converter.tplx"
tex_output_folder = "../chapters"
temp_folder = "../tmp"
xr_src_1 = "../chapters/24. Confidence Intervals - Theory"

import ntpath
tex_file = ntpath.basename(source_file).split('.ipynb')[0] + ".tex"
pdf_file = "\"" + tex_output_folder + '/' + ntpath.basename(source_file).split('.ipynb')[0] + '.pdf' + "\""

## Jinja Template for Latex

In [22]:
%%writefile $template_file

((*- extends 'article.tplx' -*))

% 1. CHANGE DOCUMENTCLASS TO STANDALONE
((* block docclass *))
\documentclass[float=false,crop=false]{standalone}
((* endblock docclass *))



% 2. REMOVE PREAMBLE AND REPLACE WITH STYLE FILE INCLUSION
((* block packages *))
\usepackage{../myipy2tex}  % NOTE WE ARE ASSSUMING THE STYLE FILE TO BE ONE FOLDER ABOVE

% if you need to cross reference to any raw tex file from this resultant tex file you  need to refer them here..
% it is not needed when you compile main.tex but make sure the labels are unique
\ifstandalone
\usepackage{xr-hyper} % Needed for external references
\externaldocument{"24. Confidence Intervals - Theory"} 
\fi


((* endblock packages *))
((* block definitions *))
((* endblock definitions *))
((* block commands *))
((* endblock commands *))

% Draw a figure using the graphicx package. Min size due to tikzmagic outputs being too shrinked.
((* macro draw_figure(filename) -*))
((* set filename = filename | posix_path *))
((*- block figure scoped -*))
    \begin{center}
    \adjustimage{max size={0.9\linewidth}{0.9\paperheight},min size={0.5\linewidth}{!}}{((( filename )))}
    \end{center}
    { \hspace*{\fill} \\}
((*- endblock figure -*))
((*- endmacro *))


% 3. BELOW STUFF ARE JUST FOR CODE CELL CORRECTIONS TO GET LINES WRAPPED PROPERLY..
%===============================================================================
% Input
%===============================================================================

% Disable input cells if there is no "show" tag, else show
((*- block input_group -*))
    ((*- if 'to_remove' in cell.metadata.get('tags', []) -*))  % remove input part of cells with tag to_remove
    %((- if cell.metadata.hide_input -))
    ((*- else -*))
        ((( custom_add_prompt(cell.source | wrap_text(88) | highlight_code(strip_verbatim=True), cell, 'In', 'incolor') )))    
    ((*- endif -*))
((*- endblock input_group -*))


%===============================================================================
% Output
%===============================================================================

% Display stream ouput with coloring
((* block stream *))
    \begin{Verbatim}[commandchars=\\\{\},fontsize=\footnotesize]
((( output.text | wrap_text(86) | escape_latex | ansi2latex )))
    \end{Verbatim}
((* endblock stream *))

%==============================================================================
% Define macro custom_add_prompt() (derived from add_prompt() macro in style_ipython.tplx)
%==============================================================================

((* macro custom_add_prompt(text, cell, prompt, prompt_color) -*))
    ((*- if cell.execution_count is defined -*))
    ((*- set execution_count = "" ~ (cell.execution_count | replace(None, " ")) -*))
    ((*- else -*))
    ((*- set execution_count = " " -*))
    ((*- endif -*))
    ((*- set indention =  " " * (execution_count | length + 7) -*))
\begin{((( prompt ~ 'Verbatim')))}[commandchars=\\\{\},fontsize=\scriptsize]
((( text | add_prompts(first='{\color{' ~ prompt_color ~ '}' ~ prompt ~ '[{\\color{' ~ prompt_color ~ '}' ~ execution_count ~ '}]:} ', cont=indention) )))
\end{((( prompt ~ 'Verbatim')))}
((*- endmacro *))

Overwriting ipy2tex_converter.tplx


## Create tex

This tex should be integrated later in main tex file as chapter or section, so shall go in to chapters folder with dependencies..

In [23]:
!jupyter nbconvert --to latex --template "$template_file" "$source_file" --output-dir="$tex_output_folder" 

[NbConvertApp] Converting notebook ../chapters/24_confidence_intervals_shallow_examples.ipynb to latex
[NbConvertApp] Support files will be in 24_confidence_intervals_shallow_examples_files\
[NbConvertApp] Making directory ../chapters\24_confidence_intervals_shallow_examples_files
[NbConvertApp] Making directory ../chapters\24_confidence_intervals_shallow_examples_files
[NbConvertApp] Making directory ../chapters\24_confidence_intervals_shallow_examples_files
[NbConvertApp] Making directory ../chapters\24_confidence_intervals_shallow_examples_files
[NbConvertApp] Making directory ../chapters\24_confidence_intervals_shallow_examples_files
[NbConvertApp] Writing 42306 bytes to ../chapters\24_confidence_intervals_shallow_examples.tex


## Test pdf

This pdf is just intermediary for us to test if output is ok (should work as we have made it 'standalone' as well). This shall also go to same chapters folder for us to refer later easily. 

In [24]:
#!pdflatex.exe -synctex=1 -interaction=nonstopmode -output-directory 1 "$tex_file"
from subprocess import Popen, PIPE
process = Popen(['pdflatex.exe', '-synctex=1', '-interaction=nonstopmode', '-output-directory', tex_output_folder, '-aux-directory', temp_folder , tex_file], shell=True, stdout=PIPE)

for line in process.stdout:
    print(line.decode().strip())

This is pdfTeX, Version 3.14159265-2.6-1.40.19 (MiKTeX 2.9.6800 64-bit)
entering extended mode

("D:/Parthiban/Business/HigherStudies/Online Courses/Udacity/Intro to Statistic
s/24. Confidence Intervals/chapters/24_confidence_intervals_shallow_examples.te
x"
LaTeX2e <2018-04-01> patch level 5
("C:\Program Files\MiKTeX 2.9\tex\latex\standalone\standalone.cls"
Document Class: standalone 2018/03/26 v1.3a Class to compile TeX sub-files stan
dalone
("C:\Program Files\MiKTeX 2.9\tex\latex\tools\shellesc.sty")
("C:\Program Files\MiKTeX 2.9\tex\generic\oberdiek\ifluatex.sty")
("C:\Program Files\MiKTeX 2.9\tex\generic\oberdiek\ifpdf.sty")
("C:\Program Files\MiKTeX 2.9\tex\generic\ifxetex\ifxetex.sty")
("C:\Program Files\MiKTeX 2.9\tex\latex\xkeyval\xkeyval.sty"
("C:\Program Files\MiKTeX 2.9\tex\generic\xkeyval\xkeyval.tex"
("C:\Program Files\MiKTeX 2.9\tex\generic\xkeyval\xkvutils.tex"
("C:\Program Files\MiKTeX 2.9\tex\generic\xkeyval\keyval.tex"))))
("C:\Program Files\MiKTeX 2.9\tex\latex\stan

("C:\Program Files\MiKTeX 2.9\tex\generic\oberdiek\ltxcmds.sty")
("C:\Program Files\MiKTeX 2.9\tex\generic\oberdiek\kvsetkeys.sty"
("C:\Program Files\MiKTeX 2.9\tex\generic\oberdiek\infwarerr.sty")
("C:\Program Files\MiKTeX 2.9\tex\generic\oberdiek\etexcmds.sty")))
("C:\Program Files\MiKTeX 2.9\tex\generic\oberdiek\pdftexcmds.sty"))
("C:\Program Files\MiKTeX 2.9\tex\latex\hyperref\hyperref.sty"
("C:\Program Files\MiKTeX 2.9\tex\generic\oberdiek\hobsub-hyperref.sty"
("C:\Program Files\MiKTeX 2.9\tex\generic\oberdiek\hobsub-generic.sty"))
("C:\Program Files\MiKTeX 2.9\tex\latex\oberdiek\auxhook.sty")
("C:\Program Files\MiKTeX 2.9\tex\latex\hyperref\pd1enc.def")
("C:\Program Files\MiKTeX 2.9\tex\latex\00miktex\hyperref.cfg")
("C:\Program Files\MiKTeX 2.9\tex\latex\url\url.sty"))
("C:\Program Files\MiKTeX 2.9\tex\latex\hyperref\hpdftex.def"
("C:\Program Files\MiKTeX 2.9\tex\latex\oberdiek\rerunfilecheck.sty"))
("C:\Program Files\MiKTeX 2.9\tex\latex\tools\longtable.sty")
("C:\Program Files

### View pdf

In [25]:
from IPython.display import HTML
def PDF(url):
    return HTML('<iframe src=%s width=700 height=350></iframe>' % url)

print('location of test pdf: ' + pdf_file)
HTML('<iframe src={0} width=100% height=600></iframe>'.format(pdf_file))

location of test pdf: "../chapters/24_confidence_intervals_shallow_examples.pdf"


## Check for Issues

I faced below issues. 
1. latex conversion issue in case of underscores [Ref](https://github.com/jupyter/nbconvert/issues/881). Tip: Always cover with braces if you have multi digit after underscore. 
2. manually created anchor links getting lost. [Ref](http://nbviewer.jupyter.org/github/rasbt/python_reference/blob/master/tutorials/table_of_contents_ipython.ipynb). Tip: Always use id name with hyfen instead of underscores and also section name linked shall be same name with space in place of hyfen.
3. Check if lists are rendered properly. Ipython is relaxed even if no end 2 spaces for each line in case of ordered/unordered lists, but latex is not. So ensure the 2 end spaces after the lines. 
4. <font color='red'>Do not use single or double quotes in comments in code cell, as that breaks latex execution later.</font>
5. Try maintaining consistency in output format. If you use print as last statement output label (eg. out\[11\]) may not appear. This may confuse users. 